In [1]:
import numpy as np
from isaacgym import gymapi, gymtorch
from isaacgym.torch_utils import *
from tqdm import tqdm

Importing module 'gym_38' (/data2/users/nk523/isaacgym/python/isaacgym/_bindings/linux-x86_64/gym_38.so)
Setting GYM_USD_PLUG_INFO_PATH to /data2/users/nk523/isaacgym/python/isaacgym/_bindings/linux-x86_64/usd/plugInfo.json
PyTorch version 2.1.1
Device count 4
/data2/users/nk523/isaacgym/python/isaacgym/_bindings/src/gymtorch


Using /data2/users/nk523/.cache/torch_extensions/py38_cu121 as PyTorch extensions root...
Emitting ninja build file /data2/users/nk523/.cache/torch_extensions/py38_cu121/gymtorch/build.ninja...
Building extension module gymtorch...
Allowing ninja to set a default number of workers... (overridable by setting the environment variable MAX_JOBS=N)
Loading extension module gymtorch...


ninja: no work to do.


## クラス構成
- MotionPrimitiveStatus class
- 

In [2]:
from franka_panda_env import FrankaPandaEnv
from motion_primitive import MotionPrimitive

In [3]:
def wrap_angle(angle):
    """Wrap an angle to the range [-pi, pi]"""
    return angle - 2 * torch.pi * torch.floor((angle + torch.pi) / (2 * torch.pi))

In [4]:
class TrajectoryCreator:
    def __init__(self, height=0.025):
        self.env = FrankaPandaEnv(use_gpu=True, use_gpu_pipeline=False)
        self.motion_primitive = MotionPrimitive(self.env.states["eef_pos"][:, :2])
        self.env.reset_pose("default")
        self.height = height
        self._move_to_default_pos()
        self._set_default_dof_pos()
        self.gain = 10.0

    def _move_to_default_pos(self):
        for _ in range(1000):
            dpose = torch.zeros((self.env.num_envs, 7))
            dpose[:, :3] = (
                torch.Tensor([0, 0, self.height]) - self.env.states["eef_pos"]
            )

            target_quat = torch.Tensor([1, 0, 0, 0])
            current_quat = self.env.states["eef_quat"]
            diff_quat = quat_mul(
                target_quat.unsqueeze(0).expand_as(current_quat),
                quat_conjugate(current_quat),
            )
            euler_angles = get_euler_xyz(diff_quat)
            dpose[:, 3], dpose[:, 4], dpose[:, 5] = (
                normalize_angle(euler_angles[0]),
                normalize_angle(euler_angles[1]),
                normalize_angle(euler_angles[2]),
            )
            dpose[:, -1] = -1

            self.env.pre_physics_step(dpose * 10)
            self.env.step_physics()
            self.env.post_physics_step()

    def _set_default_dof_pos(self):
        pose = (
            gymtorch.wrap_tensor(
                self.env.gym.acquire_dof_state_tensor(self.env.sim)
            )
            .view(self.env.num_envs, -1, 2)[:, :, 0]
            .mean(0)
            .clone()
        )
        self.env.franka_default_dof_pos = pose

    def set_force(self, dpose_xy):
        dpose = torch.zeros((self.env.num_envs, 7))
        dpose[:, :3] = torch.Tensor([0, 0, self.height]) - self.env.states["eef_pos"]
        dpose[:, :2] = dpose_xy

        target_quat = torch.Tensor([1, 0, 0, 0])
        current_quat = self.env.states["eef_quat"]
        diff_quat = quat_mul(
            target_quat.unsqueeze(0).expand_as(current_quat),
            quat_conjugate(current_quat),
        )
        euler_angles = get_euler_xyz(diff_quat)
        dpose[:, 3], dpose[:, 4], dpose[:, 5] = (
            normalize_angle(euler_angles[0]),
            normalize_angle(euler_angles[1]),
            normalize_angle(euler_angles[2]),
        )
        dpose[:, -1] = -1

        self.env.pre_physics_step(dpose * self.gain)

    def step(self):
        dpose_xy = self.motion_primitive.get_relative_goal(
            self.env.states["eef_pos"][:, :2], self.env.states["cube_pos_relative"][:, :2]
        )
        self.set_force(dpose_xy)
        print(dpose_xy)
        self.env.step_physics()
        self.env.post_physics_step()
        self.env.step_rendering()

In [5]:
trajectory_creator = TrajectoryCreator()

Not connected to PVD
+++ Using GPU PhysX
Physics Engine: PhysX
Physics Device: cuda:0
GPU Pipeline: disabled
Loading asset 'urdf/franka_description/robots/franka_panda_gripper.urdf' from 'assets'
Creating 36 environments


In [6]:
for _ in range(1000):
    trajectory_creator.step()

tensor([[ 1.0800e-01,  2.1388e-07],
        [-9.0224e-02, -1.1336e-01],
        [ 5.0069e-02,  2.7032e-02],
        [ 1.0801e-01,  2.6798e-08],
        [ 1.5801e-01,  3.8208e-07],
        [-3.9970e-02, -1.4087e-02],
        [ 1.3419e-01, -8.1096e-02],
        [-5.7481e-02, -7.5575e-02],
        [-3.9969e-02, -1.4087e-02],
        [-8.7943e-02, -2.8173e-02],
        [-9.0224e-02,  1.1336e-01],
        [ 2.8775e-02,  4.5482e-02],
        [-1.3592e-01, -4.2259e-02],
        [-8.7944e-02,  2.8174e-02],
        [-1.3342e-02, -1.4847e-01],
        [ 1.3419e-01,  8.1097e-02],
        [-8.7944e-02,  2.8173e-02],
        [ 8.8749e-04,  4.9491e-02],
        [ 5.0068e-02, -2.7032e-02],
        [-6.2258e-03,  9.8982e-02],
        [-5.7480e-02, -7.5575e-02],
        [-3.9969e-02, -1.4087e-02],
        [ 1.5800e-01, -2.1219e-07],
        [ 7.0316e-02,  1.3645e-01],
        [-1.3592e-01, -4.2260e-02],
        [ 2.8776e-02, -4.5481e-02],
        [ 4.9547e-02,  9.0963e-02],
        [ 2.8776e-02,  4.548

In [7]:
motion_primitive = MotionPrimitive(num_envs=env.num_envs)

NameError: name 'env' is not defined

In [ ]:
env.reset_pose("default")
env._refresh()
for i in tqdm(range(10000)):
    if i % 100 == 0:
        env.reset_pose()
        env.reset_cube()
    env._refresh()
    dpose = torch.zeros((env.num_envs, 7))
    dpose[:, :3] = torch.Tensor([0.0, 0.2, 0.025]) - env.states["eef_pos"]

    target_quat = torch.Tensor([1, 0, 0, 0])
    current_quat = env.states["eef_quat"]
    diff_quat = quat_mul(
        target_quat.unsqueeze(0).expand_as(current_quat),
        quat_conjugate(current_quat),
    )
    euler_angles = get_euler_xyz(diff_quat)
    dpose[:, 3], dpose[:, 4], dpose[:, 5] = (
        normalize_angle(euler_angles[0]),
        normalize_angle(euler_angles[1]),
        normalize_angle(euler_angles[2]),
    )
    dpose[:, -1] = -1

    # actions = motion_primitive.sample_actions()
    # dpose[:, :2] = actions * 0.3
    # dpose[:, 2] *= 3.0
    env.pre_physics_step(dpose * 10)
    env.step_physics()
    env.post_physics_step()
    env.step_rendering()
    # env.gym.write_viewer_image_to_file(env.viewer, f'./images/{i}.png')
    # break

In [ ]:
torch.atan2(
    env.states["cube_pos_relative"][:, 0], env.states["cube_pos_relative"][:, 1]
)

In [ ]:
env.gym.get_sim_actor_count(env.sim) // env.num_envs

In [ ]:
for i in range(1000):
    env.step_physics()
    env.step_rendering()

In [ ]:
viewer.